In [413]:
dataset, margin_rows, margin_cols, number_of_papers = bex.load_results(DATASET_FILENAME)
all_compounds = set(dataset.index.get_level_values(1))
all_activities = set(dataset.columns.get_level_values(1))
with_with_matrix = dataset.xs("w/", level=2).xs("w/", level=2, axis=1)

dataset

abiotic         \
                                                          antioxidant          
                                                                  w/o     w/   
alkaloid          acridine                            w/o      183327  64005   
                                                      w/         2475    275   
                  benzylamine                         w/o      185015  64207   
                                                      w/          787     73   
                  colchicine                          w/o      180095  64077   
...                                                               ...    ...   
terpenoid/terpene sesterterpene                       w/          186      7   
                  tetraterpene/carotenoid/xanthophyll w/o      182734  56504   
                                                      w/         3068   7776   
                  triterpene                          w/o      181206  63075   
                                                      w/         4596   1205   

                                                                         \
                                                          drought         
                                                              w/o    w/   
alkaloid          acridine                            w/o  246196  1136   
                                                      w/     2748     2   
                  benzylamine                         w/o  248084  1138   
                                                      w/      860     0   
                  colchicine                          w/o  243038  1134   
...                                                           ...   ...   
terpenoid/terpene sesterterpene                       w/      193     0   
                  tetraterpene/carotenoid/xanthophyll w/o  238462   776   
                                                      w/    10482   362   
                  triterpene                          w/o  243146  1135   
                                                      w/     5798     3   

                                                                   ...  \
                                                            metal  ...   
                                                              w/o  ...   
alkaloid          acridine                            w/o  221650  ...   
                                                      w/     2491  ...   
                  benzylamine                         w/o  223450  ...   
                                                      w/      691  ...   
                  colchicine                          w/o  218319  ...   
...                                                           ...  ...   
terpenoid/terpene sesterterpene                       w/      193  ...   
                  tetraterpene/carotenoid/xanthophyll w/o  213902  ...   
                                                      w/    10239  ...   
                  triterpene                          w/o  218392  ...   
                                                      w/     5749  ...   

                                                          pharmaco          \
                                                          sedative   wound   
                                                                w/     w/o   
alkaloid          acridine                            w/o     2503  240364   
                                                      w/         8    2671   
                  benzylamine                         w/o     2505  242195   
                                                      w/         6     840   
                  colchicine                          w/o     2477  237349   
...                                                            ...     ...   
terpenoid/terpene sesterterpene                       w/         1     191   
                  tetraterpene/carotenoid/xanthophyll w/o     2507  23229

In [414]:
def apply_metric(data, func, write=False):
    C, A = len(data.index) // 2, len(data.columns) // 2
    values = np.moveaxis(data.values.reshape((C, 2, A, 2)), 1, -2).reshape((C * A, 4))
    matrix = np.apply_along_axis(func, 1, values).reshape((C, A))
    sub = data.xs(bex.SELECTORS[True], axis=0, level=2).xs(bex.SELECTORS[True], axis=1, level=2)
    df = pd.DataFrame(matrix, index=sub.index, columns=sub.columns)
    df.index.name = "Compounds"
    df.columns.name = "Activities"

    if write:
        filename = Path(f"{DATASET_FILENAME.stem}_{func.__namme__}{DATASET_FILENAME.suffix}")
        df.to_csv(Path("results") / filename)
    return df



In [415]:
ROW_THRESHOLD = number_of_papers // 1000
COL_THRESHOLD = number_of_papers // 1000
print(f"Thresholds for rows={ROW_THRESHOLD} and cols={COL_THRESHOLD}")

rows_filter = margin_rows.xs(bex.SELECTORS[1], level=2) >= ROW_THRESHOLD
cols_filter = margin_cols.xs(bex.SELECTORS[1], level=2) >= COL_THRESHOLD
filtered = dataset.loc[rows_filter, cols_filter]
filtered.index.name = "Compounds"
filtered.columns.name = "Activities"

score_df = {}
for metric in metrics:
    metric_name = metric.__name__
    score_df[metric_name] = apply_metric(filtered, metric, False)


scores_summary_df = pd.DataFrame.from_dict(
    {f_name: [df.values.min(), df.values.mean(), df.values.max(), df.values.std()] for f_name, df in score_df.items()},
    orient="index",
    columns=["min", "mean", "max", "std"],
)

ca = catk.ca.CA(SEED)
ca.fit(score_df["tt_projection_metric"])
#display(ca.axes())
#display(ca.contributions(K = 2))

ca.contributions(K = 2)


Thresholds for rows=250 and cols=250


Coords (std)         \
                                                              1      2   
(abiotic, antioxidant)                                    1.660  0.074   
(abiotic, drought)                                        1.794 -0.211   
(abiotic, metal)                                         -0.197 -0.382   
(abiotic, salt)                                          -0.852 -1.349   
(abiotic, uv)                                             0.740 -0.177   
...                                                         ...    ...   
(terpenoid/terpene, diterpene)                           -0.576 -0.493   
(terpenoid/terpene, monoterpene)                          0.159  0.796   
(terpenoid/terpene, sesquiterpene)                       -0.490  0.132   
(terpenoid/terpene, tetraterpene/carotenoid/xan...        1.850 -0.270   
(terpenoid/terpene, triterpene)                          -0.305 -0.533   

                                                   Coords (princ.)         \
                                                                 1      2   
(abiotic, antioxidant)                                       0.797  0.026   
(abiotic, drought)                                           0.862 -0.074   
(abiotic, metal)                                            -0.095 -0.134   
(abiotic, salt)                                             -0.409 -0.474   
(abiotic, uv)                                                0.356 -0.062   
...                                                            ...    ...   
(terpenoid/terpene, diterpene)                              -0.277 -0.173   
(terpenoid/terpene, monoterpene)                             0.077  0.280   
(terpenoid/terpene, sesquiterpene)                          -0.235  0.046   
(terpenoid/terpene, tetraterpene/carotenoid/xan...           0.888 -0.095   
(terpenoid/terpene, triterpene)                             -0.146 -0.187   

                                                   Contributions (%)  ...  \
                                                                   1  ...   
(abiotic, antioxidant)                                        62.266  ...   
(abiotic, drought)                                             1.135  ...   
(abiotic, metal)                                               0.287  ...   
(abiotic, salt)                                                2.858  ...   
(abiotic, uv)                                                  1.080  ...   
...                                                              ...  ...   
(terpenoid/terpene, diterpene)                                 0.376  ...   
(terpenoid/terpene, monoterpene)                               0.007  ...   
(terpenoid/terpene, sesquiterpene)                             0.469  ...   
(terpenoid/terpene, tetraterpene/carotenoid/xan...            12.062  ...   
(terpenoid/terpene, triterpene)                                0.183  ...   

                                                   Cosine²        Quality (%)  \
                                                         1      2               
(abiotic, antioxidant)                              99.894  0.106      62.390   
(abiotic, drought)                                  99.263  0.737       1.150   
(abiotic, metal)                                    33.249 66.751       1.363   
(abiotic, salt)                                     42.703 57.297      10.027   
(abiotic, uv)                                       97.040  2.960       1.141   
...                                                    ...    ...         ...   
(terpenoid/terpene, diterpene)                      71.896 28.104       0.650   
(terpenoid/terpene, monoterpene)                     6.975 93.025       0.185   
(terpenoid/terpene, sesquiterpene)                  96.267  3.733       0.503   
(terpenoid/terpene, tetraterpene/carotenoid/xan...  98.877  1.123      12.318   
(terpenoid/terpene, triterpene)                     37.912 62.088       0.743   

                              

In [416]:

import altair as alt

source  = {
    "x": list(),
    "y": list(),
    "size":list(),
    "color": list(),
    "name": list(),
    "type": list(),
    "shape":list(),
    "croix": list(),
}




size = list(pd.concat([margin_rows[rows_filter], margin_cols[cols_filter]]).xs(bex.SELECTORS[True], level=2))
shape = ["Activity" for _ in margin_rows[rows_filter].xs(bex.SELECTORS[True], level=2)] + ["Compound" for _ in margin_cols[cols_filter].xs(bex.SELECTORS[True], level=2)]
legend_class = [name[0] for name in margin_rows[rows_filter].xs(bex.SELECTORS[True], level=2).index] + [name[0] for name in margin_cols[cols_filter].xs(bex.SELECTORS[True], level=2).index]
legend_class = list(set(legend_class))
print(legend_class)
data_select = [metric.__name__ for metric in metrics]
for metric_name in data_select:
    ca.fit(score_df[metric_name])
    current_metrix = ca.contributions(K = 2)["Coords (princ.)"]
   
    index = list(current_metrix.index)
    # index is a list of tuples get the first element and the second one in 2 different lists
    color = [i[0] for i in index]
    name = [i[1] for i in index]
    x = list(current_metrix[1])
    y = list(current_metrix[2])

    if(metric_name == "accuracy_metric"):
        x = [element * 100 for element in x]
        y = [element * 100 for element in y]


    source = {
        "x": source["x"] + x,
        "y": source["y"] + y,
        "size": source["size"] + size,
        "color": source["color"] + color,
        "name": source["name"] + name,
        "type": source["type"] + [metric_name]*len(current_metrix[1]),
        "shape": source["shape"] + shape,
        "croix": 0,
    }


input_dropdown = alt.binding_select(options=data_select, name='selection')
selection = alt.selection_single(name="selection", fields=['type'], bind=input_dropdown, init={'type': data_select[0]})
source = pd.DataFrame(source)
slider = alt.binding_range(min=0, max=max(source["size"]), step=0.1)
select_slider = alt.selection_single(name="slider_min_size", fields=['cutoff'],
                                   bind=slider, init={'cutoff': 0})

mark = alt.Chart(source).mark_point(size=60, filled=True).encode(
    alt.X('x', axis=alt.Axis(title='', grid=False)),
    alt.Y('y', axis=alt.Axis(title='', grid=False)),
    color=alt.Color('color',
        sort = alt.EncodingSortField( 'order', order = 'ascending' ),
        scale = alt.Scale(domain=legend_class),
        legend = alt.Legend(title="Color Legend")
    ),
    shape = alt.Shape('shape', scale=alt.Scale(range=['triangle-up', 'cross'])),
    size='size',
    tooltip=['name', 'color', "size", "shape"],
    
).properties(
    title='mark point',
    height = 500,
    width = 800
).interactive(
).add_selection(selection, select_slider).transform_filter(
    'selection.type == datum.type && datum.size >= slider_min_size.cutoff[0]'
)

text = alt.Chart(source).mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    x= "x",
    y= "y",
    text='name'
).transform_filter(
    'selection.type == datum.type && datum.size >= slider_min_size.cutoff[0]'
)

ruleX = alt.Chart(source).mark_rule(color='grey').encode(
    x="croix",
    size=alt.value(1)
)
ruleY = alt.Chart(source).mark_rule(color='grey').encode(
    y="croix",
    size=alt.value(2)
)



(ruleX + ruleY + mark + text).save('json_chart/mark.json')

(ruleX + ruleY + mark + text)


['phenolic compound', 'pharmaco', 'alkaloid', 'toxicity', 'terpenoid/terpene', 'abiotic', 'polyketide', 'allelopathy']


alt.LayerChart(...)

In [417]:
alt.Chart(source).mark_rect().encode(
    x='x',
    y='y',
    color='size:Q',
    tooltip=['name', 'color'],
).properties(
    height= 500
).interactive()

alt.Chart(...)

In [418]:

import altair as alt

source  = {
    "x": list(),
    "y": list(),
    "size":list(),
    "color": list(),
    "name": list(),
    "type": list(),
    "shape":list(),
    "croix": list(),
}




size = list(pd.concat([margin_rows[rows_filter], margin_cols[cols_filter]]).xs(bex.SELECTORS[True], level=2))
shape = ["Compound" for _ in margin_rows[rows_filter].xs(bex.SELECTORS[True], level=2)] + ["Activity" for _ in margin_cols[cols_filter].xs(bex.SELECTORS[True], level=2)]
legend_class = [name[0] for name in margin_rows[rows_filter].xs(bex.SELECTORS[True], level=2).index] + [name[0] for name in margin_cols[cols_filter].xs(bex.SELECTORS[True], level=2).index]
#legend_class = list(set(legend_class))
#print(legend_class)
data_select = [metric.__name__ for metric in metrics]
for metric_name in data_select:
    ca.fit(score_df[metric_name])
    current_metrix = ca.contributions(K = 2)["Coords (princ.)"]
   
    index = list(current_metrix.index)
    # index is a list of tuples get the first element and the second one in 2 different lists
    color = legend_class#[i[0] for i in index]
    name = [i[1] for i in index]
    x = list(current_metrix[1])
    y = list(current_metrix[2])

    if(metric_name == "accuracy_metric"):
        x = [element * 100 for element in x]
        y = [element * 100 for element in y]


    source = {
        "x": source["x"] + x,
        "y": source["y"] + y,
        "size": source["size"] + size,
        "color": source["color"] + color,
        "name": source["name"] + name,
        "type": source["type"] + [metric_name]*len(current_metrix[1]),
        "shape": source["shape"] + shape,
        "croix": 0,
    }

selection_legend = alt.selection_multi(fields=['color'])
color = alt.condition(selection_legend,
                      alt.Color('color:O', legend=None, 
                      scale=alt.Scale(scheme='category10')),
                      alt.value('lightgray'))

input_dropdown = alt.binding_select(options=data_select, name='selection')
selection = alt.selection_single(name="selection", fields=['type'], bind=input_dropdown, init={'type': data_select[0]})
source = pd.DataFrame(source)
slider = alt.binding_range(min=0, max=max(source["size"]), step=0.1)
select_slider = alt.selection_single(name="slider_min_size", fields=['cutoff'],
                                   bind=slider, init={'cutoff': 0})

mark = alt.Chart(source).mark_point(size=60, filled=True).encode(
    alt.X('x', axis=alt.Axis(title='', grid=False)),
    alt.Y('y', axis=alt.Axis(title='', grid=False)),
    color = color,
    shape = alt.Shape('shape',
        scale = alt.Scale(domain=["Activity", "Compound"], range=['triangle-up', 'cross']),
        legend = alt.Legend(title="Color Legend")
        ),
    #alt.Shape('shape', scale=alt.Scale(range=['triangle-up', 'cross'])),
    size='size',
    tooltip=['name', 'color', "size", "shape"],
    
).properties(
    title='mark point',
    height = 500,
    width = 800
).interactive(
).add_selection(selection, select_slider).transform_filter(
    'selection.type == datum.type && datum.size >= slider_min_size.cutoff[0]'
)

text = alt.Chart(source).mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    x= "x",
    y= "y",
    color= alt.condition(selection_legend,
                      alt.value('black'),
                      alt.value('')),
    text='name'
).add_selection(selection_legend
).transform_filter(
    'selection.type == datum.type && datum.size >= slider_min_size.cutoff[0]'
)

ruleX = alt.Chart(source).mark_rule(color='grey').encode(
    x="croix",
    size=alt.value(1)
)
ruleY = alt.Chart(source).mark_rule(color='grey').encode(
    y="croix",
    size=alt.value(2)
)

legend = alt.Chart(source
).mark_point(filled=True, size=250
).encode(
    y=alt.Y('color', axis=alt.Axis(orient='right')),
    shape="shape",
    color=color
).add_selection(
selection_legend
)

descriptive_text = alt.Chart().mark_text(align='left').encode(
    
    y= alt.value(222),
    text=alt.value(["Click on the shapes of the legend or the graphic to select an item.","",
                    "Shift + click to select multiple legend items.","",
                    "Double click on the legend to select all the items."]),
    size=alt.value(12)
)


((ruleX + ruleY + mark + text) | (legend + descriptive_text)).save('json_chart/mark.json')
